In [5]:
from langchain_ollama  import OllamaLLM
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
import pandas as pd
import numpy as np
import chromadb
from typing import List, Dict, Any
import re
from sentence_transformers import SentenceTransformer
from app.utils.functions import *
from app.core import config
from IPython.display import HTML
from app.services.ollama_service import format_response
import httpx

d:\SL_CB_IA\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocessing_data(df: pd.DataFrame, simple_dict: list[dict]) -> pd.DataFrame:
    """Prétraite et catégorise le DataFrame selon les groupes définis."""
    try:
        required_columns = ['Lignes', 'Contexte', 'Nature de l\'écriture', 'Année', 'Mois', 'Montant']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"La colonne requise '{col}' est absente du DataFrame.")
        mask = (
            (df.iloc[:, 0] == df.iloc[0, 0]) &
            (df.iloc[:, 2] == "Compte d'exploitation") &
            (df.iloc[:, 8] != "Colonne variation")
        )
        df_filtered: pd.DataFrame
        df_filtered = df[mask].copy()
        
        if df_filtered.empty:
            df_filtered['Groupe'] = pd.Series(dtype='object')
            return df_filtered

        mask_pct_recettes = df_filtered['Lignes'] == "% DES RECETTES TOTALES"
        df_pct_recettes = df_filtered[mask_pct_recettes].copy()
        df_autres = df_filtered[~mask_pct_recettes].copy()

        if not df_pct_recettes.empty:
            sort_keys = ['Année', 'Mois', 'Contexte', "Nature de l'écriture", 'Montant']
            existing_sort_keys = [col for col in sort_keys if col in df_pct_recettes.columns]
            df_pct_recettes = (
                df_pct_recettes
                .sort_values(by=existing_sort_keys[:-1] + ['Montant'], ascending=[True]*len(existing_sort_keys[:-1])+[False])
                .reset_index(drop=True)
            )
            code_map = {0: "4.", 1: "7.", 2: "10."}
            df_pct_recettes['Code Hiérarchique'] = (
                df_pct_recettes
                .groupby(existing_sort_keys[:-1], sort=False)
                .cumcount()
                .map(lambda idx: code_map.get(idx, None))
            )

        coded_tree = generate_hierarchy_codes(simple_dict)
        hierarchy_list = extract_flat_hierarchy_list(coded_tree)
        mapping_dict = {item['label']: item['code'] for item in hierarchy_list}

        if not df_autres.empty:
            df_autres['Code Hiérarchique'] = df_autres['Lignes'].map(mapping_dict)
        df_filtered = pd.concat([df_autres, df_pct_recettes]).sort_index(kind="stable")
        cols = list(df.columns)
        if "Code Hiérarchique" in cols and "Lignes" in cols:
            cols.remove("Code Hiérarchique")
            insert_idx = cols.index("Lignes")
            cols = cols[:insert_idx] + ["Code Hiérarchique"] + cols[insert_idx:]
            df_filtered = df_filtered[cols]

        result_list = extract_all_descendants_for_list(simple_dict)
        CHIFFRE_AFFAIRES = result_list[0] + result_list[3] + result_list[6] + result_list[9] + result_list[10]
        CHARGES = result_list[1] + result_list[4] + result_list[7] + result_list[12]
        MARGES = result_list[2] + result_list[5] + result_list[8] + result_list[11] + result_list[13]
        groupes_dict = {
            "Chiffre d'affaire": CHIFFRE_AFFAIRES,
            "Charge": CHARGES,
            "Marge": MARGES
        }
        groupes_mapping = {poste: groupe for groupe, postes in groupes_dict.items() for poste in postes}
        df_filtered['Groupe'] = df_filtered['Lignes'].map(groupes_mapping)

        mask_pct = df_filtered['Lignes'] == "% DES RECETTES TOTALES"
        mask_non_pct = ~mask_pct

        # Correction : bien gérer le typage et l'arrondi sans convertir en int les NaN et préserver les floats si NaN
        df_filtered.loc[mask_non_pct, "Montant"] = df_filtered.loc[mask_non_pct, "Montant"].round(0)
        mask_non_pct_non_nan = mask_non_pct & df_filtered['Montant'].notna()
        df_filtered.loc[mask_non_pct_non_nan, "Montant"] = df_filtered.loc[mask_non_pct_non_nan, "Montant"].astype(int)

        if mask_pct.any():
            df_filtered.loc[mask_pct, "Montant"] = df_filtered.loc[mask_pct, "Montant"].round(2)
        
        df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)
        return df_filtered
    except Exception as e:
        logger.error(f"Erreur lors du prétraitement des données : {e}")
        raise

In [3]:
# Chargement du fichier (en supposant la même structure que précédemment)
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
lexiques = await get_mapping()


df = pd.read_csv(r"data.csv")
df = preprocessing_data(df, simple_dict)

# Renommage et nettoyage
df = df.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Contexte': 'Contexte',
        'Année': 'Annee',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df['Annee'] = df['Annee'].astype(int)
df['Mois'] = df['Mois'].astype(int)
df['Contexte'] = df['Contexte'].replace({'R': 'Réel', 'B': 'Budget', 'P': 'Prévision'})

df_agg = df.groupby(
    [
        'Residence', 'Colonnes', 'Annee', 'Mois', "Nature de l'écriture", 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'
    ]
)['Montant'].sum().reset_index()

In [4]:
contexte_order = ['Réel', 'Prévision', 'Budget']

# Ajoute 'Nature de l\'écriture' dans les colonnes du pivot (et pas dans l'index)
df_pivot = df_agg.pivot_table(
    index=['Groupe', 'Code_H', 'Ligne_Analytique'],
    columns=['Annee', 'Contexte', 'Mois', "Nature de l'écriture"],
    values='Montant',
    fill_value=0,
    aggfunc='sum'
)

if df_pivot.columns.nlevels == 4:
    nature_order_desc = sorted(df_pivot.columns.get_level_values(3).unique(), reverse=True)
    nature_order_dict = {name: i for i, name in enumerate(nature_order_desc)}
    df_pivot = df_pivot[
        sorted(
            df_pivot.columns,
            key=lambda x: (
                x[0],  # Annee
                contexte_order.index(x[1]) if x[1] in contexte_order else 99,  # Contexte, custom order
                contexte_order.index(x[2]) if x[2] in contexte_order else 99,  # Mois (ordre du contexte)
                nature_order_dict.get(x[3], 999)  # Nature de l'écriture, descending
            )
        )
    ]

df_pivot = df_pivot.reset_index()
df_pivot

Annee                 Groupe   Code_H                   Ligne_Analytique  \
Contexte                                                                   
Mois                                                                       
Nature de l'écriture                                                       
0                     Charge      13.                              CAPEX   
1                     Charge       2.        CHARGES D'IMMEUBLE DIRECTES   
2                     Charge     2.1.                 FRAIS DE PERSONNEL   
3                     Charge   2.1.1.                           Salaires   
4                     Charge  2.1.10.  Prestataires sécurité gardiennage   
..                       ...      ...                                ...   
146                    Marge      12.                             EBITDA   
147                    Marge      14.                     FREE CASH FLOW   
148                    Marge       3.                            MARGE 1   
149                    Marge       6.                            MARGE 2   
150                    Marge       9.                            MARGE 3   

Annee                     2022                                          \
Contexte                  Réel                                           
Mois                         1         2         3         4         5   
Nature de l'écriture Mensuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                          0.0       0.0       0.0   16774.0    5431.0   
1                     113102.0   25643.0   95519.0   62063.0   23530.0   
2                       8556.0   10998.0   19345.0   11742.0   11108.0   
3                       4146.0    4170.0    4254.0    4633.0    3908.0   
4                       -112.0       0.0       0.0       0.0       0.0   
..                         ...       ...       ...       ...       ...   
146                    39274.0  -36423.0  -53258.0   78695.0  -39898.0   
147                    39274.0  -36423.0  -53258.0   61921.0  -45329.0   
148                    53541.0  116352.0   43904.0   95627.0  109683.0   
149                    43339.0  -36559.0  -28520.0   85447.0  -40515.0   
150                    38426.0  -37189.0  -54106.0   77874.0  -40760.0   

Annee                                     ...      2026                      \
Contexte                                  ...    Budget                       
Mois                         6         7  ...         4         5         6   
Nature de l'écriture Mensuelle Mensuelle  ... Mensuelle Mensuelle Mensuelle   
0                          0.0   41603.0  ...       0.0       0.0       0.0   
1                      43562.0   68310.0  ...   57354.0   25080.0   42573.0   
2                      14427.0   16407.0  ...   12274.0    9754.0   12896.0   
3                       4698.0    4698.0  ...    6677.0    6677.0    6677.0   
4                          0.0     -55.0  ...       0.0       0.0    -309.0   
..                         ...       ...  ...       ...       ...       ...   
146                    -6439.0   79330.0  ...  110156.0  116129.0 -119385.0   
147                    -6439.0   37727.0  ...  110156.0  116129.0 -119385.0   
148                    93433.0  107068.0  ...  147603.0  146724.0  133827.0   
149                    18937.0   94294.0  ...  128863.0  127934.0 -108820.0   
150                    -7305.0   78095.0  ...  107481.0  113382.0 -122061.0   

Annee                                                                   \
Contexte                                                                 
Mois                         7         8         9        10        11   
Nature de l'écriture Mensuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                          0.0       0.0       0.0       0.0       0.0   
1                      63333.0   29281.0   34167.0   60469.0   31570.0   
2                      15299.0   13764.0    9544.0   10331.0    9880.0   
3                       6677.0    6677.0    

In [5]:
def code_hierarchical_sort_key(code):
    parts = str(code).strip('.').split('.')
    return [int(part) if part.isdigit() else part for part in parts if part]

df_pivot_sorted = df_pivot.copy()
df_pivot_sorted['__sort_key'] = df_pivot_sorted['Code_H'].apply(code_hierarchical_sort_key)
sorted_indexes = df_pivot_sorted.sort_values('__sort_key').index
df_pivot_sorted = df_pivot_sorted.loc[sorted_indexes].drop(columns='__sort_key').reset_index(drop=True)

df_pivot_sorted.to_csv("df_pivot.csv")
df_pivot_sorted

C:\Users\Amsata\AppData\Local\Temp\ipykernel_20048\3123885414.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_pivot_sorted = df_pivot_sorted.loc[sorted_indexes].drop(columns='__sort_key').reset_index(drop=True)


Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire      1.   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
146                   Chiffre d'affaire     10.   
147                   Chiffre d'affaire     11.   
148                               Marge     12.   
149                              Charge     13.   
150                               Marge     14.   

Annee                                     Ligne_Analytique       2022  \
Contexte                                                         Réel   
Mois                                                                1   
Nature de l'écriture                                        Mensuelle   
0                                                 RECETTES  166643.00   
1                          Loyers logements et parkings HT  143177.00   
2                                     CA Locatif Estudines  142098.00   
3                                      CA Locatif Parkings    1078.00   
4                                         RECETTES ANNEXES   23467.00   
..                                                     ...        ...   
146                                 % DES RECETTES TOTALES      23.06   
147                   Dotations aux amortissements (CAPEX)     849.00   
148                                                 EBITDA   39274.00   
149                                                  CAPEX       0.00   
150                                         FREE CASH FLOW   39274.00   

Annee                                                                       \
Contexte                                                                     
Mois                          2          3          4         5          6   
Nature de l'écriture  Mensuelle  Mensuelle  Mensuelle Mensuelle  Mensuelle   
0                     141995.00  139423.00  157690.00  133213.0  136995.00   
1                     140713.00  140431.00  130839.00  130957.0  132729.00   
2                     139996.00  139868.00  130404.00  130598.0  132427.00   
3                        716.00     564.00     435.00     359.0     302.00   
4                       1282.00      38.00   26850.00    2256.0    4657.00   
..                          ...        ...        ...       ...        ...   
146                      -26.19     -38.81      49.38     -30.6      -5.33   
147                      766.00     848.00     821.00     862.0     866.00   
148                   -36423.00  -53258.00   78695.00  -39898.0   -6439.00   
149                        0.00       0.00   16774.00    5431.0       0.00   
150                   -36423.00  -53258.00   61921.00  -45329.0   -6439.00   

Annee                            ...       2026                                \
Contexte                         ...     Budget                                 
Mois                          7  ...          4         5         6         7   
Nature de l'écriture  Mensuelle  ...  Mensuelle Mensuelle Mensuelle Mensuelle   
0                     175378.00  ...  204957.00  171803.0  176400.0  204364.0   
1                     133824.00  ...  168918.00  168918.0  172014.0  168479.0   
2                     133522.00  ...  168318.00  168318.0  171414.0  167879.0   
3                        302.00  ...     600.00     600.0     600.0     600.0   
4                      41555.00  ...   36150.00    3000.0    4500.0   36000.0   
..                          ...  ...        ...       ...       ...       ...   
146                       44.53  ...      52.44      66.0     -69.2      49.0   
147                     1234.00  ...    2675.00    2747.0    2675.0    3116.

In [6]:
mask_annuelle = df_pivot_sorted.columns.get_level_values(3) == "Annuelle"
annuelle_cols = df_pivot_sorted.columns[mask_annuelle].tolist()

meta_cols = [c for c in df_pivot_sorted.columns if c[0] in ("Groupe", "Code_H", "Ligne_Analytique")]

selected_cols = meta_cols + annuelle_cols
df_pivot_sorted_annual = df_pivot_sorted.loc[:, selected_cols]

df_pivot_sorted_annual.to_csv("df_pivot_sorted_annual.csv")
df_pivot_sorted_annual

Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire      1.   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
146                   Chiffre d'affaire     10.   
147                   Chiffre d'affaire     11.   
148                               Marge     12.   
149                              Charge     13.   
150                               Marge     14.   

Annee                                     Ligne_Analytique        2022  \
Contexte                                                          Réel   
Mois                                                                12   
Nature de l'écriture                                          Annuelle   
0                                                 RECETTES  1856015.00   
1                          Loyers logements et parkings HT  1715744.00   
2                                     CA Locatif Estudines  1709962.00   
3                                      CA Locatif Parkings     5782.00   
4                                         RECETTES ANNEXES   129946.00   
..                                                     ...         ...   
146                                 % DES RECETTES TOTALES        2.11   
147                   Dotations aux amortissements (CAPEX)    14788.00   
148                                                 EBITDA    54004.00   
149                                                  CAPEX    95452.00   
150                                         FREE CASH FLOW   -41447.00   

Annee                       2023        2024        2025              \
Contexte                    Réel        Réel   Prévision      Budget   
Mois                          12          12          12          12   
Nature de l'écriture    Annuelle    Annuelle    Annuelle    Annuelle   
0                     2065453.00  2193409.00  2182306.00  2205868.00   
1                     1908910.00  2026364.00  2023357.00  2038521.00   
2                     1903970.00  2019099.00  2017043.00  2031296.00   
3                        4940.00     7265.00     6315.00     7224.00   
4                      182250.00   203148.00   171106.00   179398.00   
..                           ...         ...         ...         ...   
146                        12.11       10.22        8.33       10.08   
147                     25150.00    28095.00    23186.00    27076.00   
148                    275367.00   252303.00   204917.00   249344.00   
149                     34342.00    15964.00    19270.00        0.00   
150                    241025.00   236339.00   185646.00   249344.00   

Annee                      2026  
Contexte                 Budget  
Mois                         12  
Nature de l'écriture   Annuelle  
0                     2221683.0  
1                     2057395.0  
2                     2050195.0  
3                        7200.0  
4                      176470.0  
..                          ...  
146                        13.4  
147                     34447.0  
148                    332103.0  
149                         0.0  
150                    332103.0  

[151 rows x 9 columns]

In [130]:
# Retirer la colonne "Code_H" (représentée sous forme de tuple dans le MultiIndex colonne)
df_h1 = df_pivot_sorted_annual[df_pivot_sorted_annual["Code_H"].apply(lambda x: len(str(x).strip('.').split('.')) < 2)].reset_index(drop=True)
if ("Code_H", "", "", "") in df_h1.columns:
    df_h1 = df_h1.drop(columns=[("Code_H", "", "", "")])

# Afficher uniquement les 3 dernières années
# Identifier toutes les années disponibles dans le MultiIndex colonne (hors les colonnes méta)
years = sorted({c[0] for c in df_h1.columns if isinstance(c, tuple) and isinstance(c[0], (int, float))})
last3_years = years[-3:] if len(years) >= 3 else years

meta_cols = [c for c in df_h1.columns if c[0] in ("Groupe", "Ligne_Analytique")]
annual_cols_last3 = [c for c in df_h1.columns if isinstance(c, tuple) and c[0] in last3_years]
cols_to_keep = meta_cols + annual_cols_last3
df_pivot_sorted_annual_h1 = df_h1.loc[:, cols_to_keep]

print(df_pivot_sorted_annual_h1.to_markdown(index = False))

| ('Groupe', '', '', '')   | ('Ligne_Analytique', '', '', '')     |   (2024, 'Réel', 12, 'Annuelle') |   (2025, 'Prévision', 12, 'Annuelle') |   (2025, 'Budget', 12, 'Annuelle') |   (2026, 'Budget', 12, 'Annuelle') |
|:-------------------------|:-------------------------------------|---------------------------------:|--------------------------------------:|-----------------------------------:|-----------------------------------:|
| Chiffre d'affaire        | RECETTES                             |                      2.19341e+06 |                           2.18231e+06 |                        2.20587e+06 |                        2.22168e+06 |
| Charge                   | CHARGES D'IMMEUBLE DIRECTES          |                 663916           |                      666871           |                   650619           |                   602278           |
| Marge                    | MARGE 1                              |                      1.52949e+06 |                           1.5

In [8]:
df_pivot_sorted_annual_h2 = df_pivot_sorted_annual[df_pivot_sorted_annual["Code_H"].apply(lambda x: len(str(x).strip('.').split('.')) < 3)].reset_index(drop=True)

print(df_pivot_sorted_annual_h2.to_markdown(index = False))

| ('Groupe', '', '', '')   | ('Code_H', '', '', '')   | ('Ligne_Analytique', '', '', '')             |   (2022, 'Réel', 12, 'Annuelle') |   (2023, 'Réel', 12, 'Annuelle') |   (2024, 'Réel', 12, 'Annuelle') |   (2025, 'Prévision', 12, 'Annuelle') |   (2025, 'Budget', 12, 'Annuelle') |   (2026, 'Budget', 12, 'Annuelle') |
|:-------------------------|:-------------------------|:---------------------------------------------|---------------------------------:|---------------------------------:|---------------------------------:|--------------------------------------:|-----------------------------------:|-----------------------------------:|
| Chiffre d'affaire        | 1.                       | RECETTES                                     |                      1.85602e+06 |                      2.06545e+06 |                      2.19341e+06 |                           2.18231e+06 |                        2.20587e+06 |                        2.22168e+06 |
| Chiffre d'affaire        | 1.1. 

---

In [9]:
# ---------------------------
# Helpers pour récupérer valeur depuis MultiIndex
# ---------------------------
def get_value_for_year(
    df_pivot: pd.DataFrame, 
    row_idx: int, 
    year: int,
    contexte_preference: List[str] = None,
    month: int = 12,
    nature: str = "Annuelle"
):
    """
    Récupère la valeur dans df_pivot pour la ligne `row_idx` et la colonne (year, contexte, month, nature).
    contexte_preference: ordre de préférence, ex ["Réel","Budget","Prévision"]
    Retourne (valeur, contexte_found) ou (np.nan, None)
    """
    cols = df_pivot.columns
    if contexte_preference is None:
        contexte_preference = sorted({c[1] for c in cols if not (isinstance(c, str))})
    for contexte in contexte_preference:
        target = (year, contexte, month, nature)
        if target in cols:
            return df_pivot.iloc[row_idx][target], contexte
        cand_cols = [c for c in cols if isinstance(c, tuple) and c[0] == year and c[1] == contexte and c[3] == nature]
        exact = [c for c in cand_cols if c[2] == month]
        if exact:
            return df_pivot.iloc[row_idx][exact[0]], contexte
        if cand_cols:
            return df_pivot.iloc[row_idx][cand_cols[0]], contexte
    cand = [c for c in cols if isinstance(c, tuple) and c[0] == year and c[3] == nature]
    if cand:
        return df_pivot.iloc[row_idx][cand[0]], cand[0][1]
    return np.nan, None

def safe_isfinite(x):
    # Only numeric types, not bool or complex
    if isinstance(x, (int, float, np.float64, np.integer, np.floating)) and not isinstance(x, bool):
        return np.isfinite(x)
    else:
        return False

# ---------------------------
# Fonctions de génération de chunks
# ---------------------------
def make_line_cell_chunk(
    df_pivot: pd.DataFrame, 
    row_idx: int, 
    year: int,
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Crée un chunk correspondant à la cellule annuelle pour une ligne x année (type: LINE_CELL).
    """
    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    
    val, contexte_used = get_value_for_year(df_pivot, row_idx, year, contexte_preference)
    # Use safe_isfinite for robust numeric check
    val_str = "N/A"
    if pd.notna(val) and safe_isfinite(val):
        # check if ligne starts with %
        if isinstance(ligne, str) and ligne.strip().startswith("%"):
            val_str = f"{val:.2f} %"
        else:
            val_str = f"{val:.0f} €"

    text = (
        f"[TYPE: LINE_CELL]\n"
        f"Résidence: {residence_label}\n"
        f"Année: {year}\n"
        f"Contexte: {contexte_used or 'N/A'}\n"
        f"Groupe: {groupe}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Ligne analytique: {ligne}\n"
        f"Valeur annuelle: {val_str}\n"
    )
    metadata = {
        "chunk_type": "LINE_CELL",
        "residence": residence_label,
        "year": int(year),
        "contexte": contexte_used,
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne,
    }
    return {"text": text, "metadata": metadata}

def make_annual_summary_chunk(
    df_pivot: pd.DataFrame, row_idx: int, year: int,
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Synthèse textuelle d'une ligne (ligne analytique) pour une année (type: ANNUAL_SUMMARY).
    Contient valeur, variation vs previous year (si disponible), pct change if possible.
    """
    val_cur, ctx = get_value_for_year(df_pivot, row_idx, year, contexte_preference)
    val_prev, _ = get_value_for_year(df_pivot, row_idx, year-1, contexte_preference)
    # formatting
    def fmt(val, ligne):
        if pd.notna(val) and safe_isfinite(val):
            # check if ligne starts with %
            if isinstance(ligne, str) and ligne.strip().startswith("%"):
                return f"{val:.2f} %"
            else:
                return f"{val:.0f} €"
        else:
            return "N/A"

    def pct_change(a, b):
        try:
            if pd.isna(a) or pd.isna(b) or not safe_isfinite(a) or not safe_isfinite(b) or b == 0:
                return None
            return (a - b) / abs(b) * 100
        except Exception:
            return None

    change_pct = pct_change(val_cur, val_prev)
    if (change_pct is not None and isinstance(change_pct, (int, float, np.float64)) and safe_isfinite(change_pct)):
        change_str = f"{change_pct:.2f}% vs {year-1}"
    else:
        change_str = None

    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    text = (
        f"[TYPE: ANNUAL_SUMMARY]\n"
        f"Résidence: {residence_label}\n"
        f"Ligne: {ligne}\n"
        f"Année: {year}\n"
        f"Contexte utilisé: {ctx or 'N/A'}\n"
        f"Valeur {year}: {fmt(val_cur, ligne)}\n"
        f"Valeur {year-1}: {fmt(val_prev, ligne)}\n"
        f"Variation: {change_str or 'N/A'}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Groupe: {groupe}\n"
    )
    metadata = {
        "chunk_type": "ANNUAL_SUMMARY",
        "residence": residence_label,
        "year": int(year),
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne
    }
    return {"text": text, "metadata": metadata}

def make_trend_chunk(
    df_pivot: pd.DataFrame, row_idx: int, years: List[int],
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Trend chunk over multiple years (type: TREND_MULTI_YEAR).
    years: list[int] e.g. [2022,2023,2024]
    """
    values = []
    used_ctx = None
    for y in years:
        v, ctx = get_value_for_year(df_pivot, row_idx, y, contexte_preference)
        if pd.notna(v) and safe_isfinite(v):
            values.append(v)
        else:
            values.append(None)
        if used_ctx is None and ctx is not None:
            used_ctx = ctx
    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    
    def fmt(val):
        if pd.notna(val) and safe_isfinite(val):
            # check if ligne starts with %
            if isinstance(ligne, str) and ligne.strip().startswith("%"):
                return f"{val:.2f} %"
            else:
                return f"{val:.0f} €"
        else:
            return "N/A"

    vals_str = ", ".join([f"{y}: {fmt(v)}" for y, v in zip(years, values)])
    first = values[0] if values else None
    last = values[-1] if values else None
    pct_total = None
    if first not in (None, 0) and last is not None and safe_isfinite(first) and safe_isfinite(last):
        try:
            pct_total = (last - first) / abs(first) * 100
        except Exception:
            pct_total = None
    pct_str = f"{pct_total:.2f}% ({years[0]}→{years[-1]})" if pct_total is not None and safe_isfinite(pct_total) else "N/A"
    text = (
        f"[TYPE: TREND_MULTI_YEAR]\n"
        f"Résidence: {residence_label}\n"
        f"Ligne: {ligne}\n"
        f"Années: {years}\n"
        f"Contexte principal trouvé: {used_ctx or 'N/A'}\n"
        f"Valeurs: {vals_str}\n"
        f"Variation globale: {pct_str}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Groupe: {groupe}\n"
    )
    metadata = {
        "chunk_type": "TREND_MULTI_YEAR",
        "residence": residence_label,
        "years": [int(y) for y in years],
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne
    }
    return {"text": text, "metadata": metadata}

def make_recommendation_chunk(
    df_pivot: pd.DataFrame,
    row_idx: int,
    years: List[int],
    residence_label: str,
    contexte_preference: List[str] = None):
    """
    Template de recommandations automatiques (type: RECOMMENDATION) basé sur trend simple.
    Peut être enrichi par des heuristiques supplémentaires.
    Utilise le chunk tendance calculé pour robustesse et éventuelle extension.
    """
    if contexte_preference is None:
        contexte_preference = ["Réel", "Prévision", "Budget"]

    # Récupération du chunk de tendance pour contexte et robustesse
    trend = make_trend_chunk(df_pivot, row_idx, years, residence_label, contexte_preference)
    # On va utiliser l'analyse de la tendance issue du chunk trend
    # On exploite pct_total si possible

    trend_metadata = trend.get("metadata", {})
    values = []
    for y in years:
        v, _ = get_value_for_year(df_pivot, row_idx, y, contexte_preference)
        values.append(v if (pd.notna(v) and safe_isfinite(v)) else None)
    first = values[0] if values else None
    last = values[-1] if values else None

    # Essaie d'utiliser le pourcentage trend du chunk trend, sinon recalcule localement
    pct_total = None
    trend_text = trend.get("text", "")
    try:
        # Recherche d'une ligne contenant "Variation globale: x%"
        m = re.search(r"Variation globale:\s*([-\d.,]+)%", trend_text)
        if m:
            pct_total = float(m.group(1).replace(',', '.'))
    except Exception:
        pct_total = None

    if pct_total is None and first not in (None, 0) and last is not None and safe_isfinite(first) and safe_isfinite(last):
        try:
            pct_total = (last - first) / abs(first) * 100
        except Exception:
            pct_total = None

    rec = []
    if pct_total is not None and safe_isfinite(pct_total):
        if pct_total > 10:
            rec.append(
                "Tendance fortement positive → vérifier si le budget sous-estime la croissance ; envisager de réallouer le budget investissement."
            )
        elif pct_total < -10:
            rec.append(
                "Tendance négative → investiguer les causes (baisse du CA / hausse des charges)."
            )
        else:
            rec.append("Tendance stable → maintenir le monitoring mensuel.")
    else:
        rec.append("Données insuffisantes pour recommandations automatiques.")

    text = (
        f"[TYPE: RECOMMENDATION]\n"
        f"Résidence: {residence_label}\n"
        f"Années examinées: {years}\n"
        f"Variation détectée (extrait trend): "
        f"{pct_total:.2f}%\n" if pct_total is not None and safe_isfinite(pct_total) else
        f"Variation détectée: N/A\n"
    )
    text += f"Recommandations:\n- " + "\n- ".join(rec) + "\n"

    metadata = {
        "chunk_type": "RECOMMENDATION",
        "residence": residence_label,
        "years": [int(y) for y in years],
        "trend_metadata": trend_metadata
    }
    return {"text": text, "metadata": metadata}

# ---------------------------
# Pipeline principal : génération de tous les chunks
# ---------------------------
def generate_chunks_from_pivot(
    df_pivot: pd.DataFrame,
    residence_label: str,
    years_for_trend: List[int] = None,
    contexte_preference: List[str] = None) -> pd.DataFrame:
    """
    Parcourt chaque ligne (Ligne_Analytique) du df_pivot et génère :
        - ligne-cell chunks pour chaque year in columns (Annuelle)
        - annual summary chunks
        - trend chunks (par défaut pour dernières 3 années trouvées)
        - recommendation chunk
    Retourne un DataFrame chunks_df avec colonnes: text, metadata (dict)
    """
    if years_for_trend is None:
        years = sorted({c[0] for c in df_pivot.columns if isinstance(c, tuple) and isinstance(c[0], (int, np.integer))})
        years_for_trend = years[-3:]
    if contexte_preference is None:
        contexte_preference = ["Réel", "Prévision", "Budget"]

    chunks: List[Dict[str, Any]] = []
    n_rows = df_pivot.shape[0]
    for i in range(n_rows):
        row = df_pivot.iloc[i]
        ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
        if (isinstance(ligne, str) and ligne.strip() == "") or (not isinstance(ligne, str) and (pd.isna(ligne) if not isinstance(ligne, pd.Series) else ligne.isna().all())):
            continue
        years_all = sorted({c[0] for c in df_pivot.columns if isinstance(c, tuple) and c[3] == "Annuelle" and isinstance(c[0], (int, np.integer))})
        for y in years_all:
            chunks.append(make_line_cell_chunk(df_pivot, i, y, residence_label, contexte_preference))
            chunks.append(make_annual_summary_chunk(df_pivot, i, y, residence_label, contexte_preference))
        if len(years_for_trend) >= 2:
            chunks.append(make_trend_chunk(df_pivot, i, years_for_trend, residence_label, contexte_preference))
            chunks.append(make_recommendation_chunk(df_pivot, i, years_for_trend, residence_label))

    chunks_df = pd.DataFrame([{
        "text": c["text"],
        **{f"meta_{k}": (v if not isinstance(v, (list, dict)) else str(v)) for k, v in c["metadata"].items()}
    } for c in chunks])
    return chunks_df

# ---------------------------
# Utilisation (exemple)
# ---------------------------
# df_pivot_sorted : ton DataFrame pivot affiché (index numérique, colonnes MultiIndex)
# residence_label : "11000099 - Noisy" (ou simplement "Noisy")
# years_for_trend : par défaut dernières 3 années disponibles

# Exemple d'appel :
chunks_df = generate_chunks_from_pivot(df_pivot_sorted, residence_label="Noisy")
chunks_df.to_csv("chunks_from_pivot.csv", index=False)

In [10]:
chunks_df.shape

(1812, 10)

In [11]:
print(
    chunks_df
    .loc[
        (chunks_df["meta_ligne"] == "% DES RECETTES TOTALES") & 
        (chunks_df["meta_chunk_type"] == "LINE_CELL"),
        "text"
    ]
    .iloc[0]
)

[TYPE: LINE_CELL]
Résidence: Noisy
Année: 2022
Contexte: Réel
Groupe: Chiffre d'affaire
Code Hiérarchique: 4.
Ligne analytique: % DES RECETTES TOTALES
Valeur annuelle: 63.65 %



---

In [12]:
# ------------------------
# CHUNKS PAR LIGNE ANNUELLE
# ------------------------
chunks_by_lines_annual = []
cols = sorted(set([col for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple) and col[-1] == "Annuelle"]))
for _, row in df_pivot_sorted.iterrows():
    doc = {
        "groupe": row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"],
        "code_h": row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"],
        "ligne": row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"],
        "valeurs": (
            {
                (
                    f"{c[1]} {c[3]} {c[0]}"
                ):
                (
                    "{:.0f} €".format(v.iloc[0] if hasattr(v, "iloc") else v)
                    if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                    else "N/A"
                )
                for c in cols
                for v in [row.get(c)]
            }
            if not str(row["Ligne_Analytique"].iloc[-1]).lstrip().startswith('%')
            else
            {
                (
                    f"{c[1]} {c[3]} {c[0]}"
                ):
                (
                    "{:.2f} %".format(v.iloc[0] if hasattr(v, "iloc") else v)
                    if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                    else "N/A"
                )
                for c in cols
                for v in [row.get(c)]
            }
        )
    }
    chunks_by_lines_annual.append(doc)

chunks_by_lines_annual_df = pd.DataFrame(chunks_by_lines_annual)
chunks_by_lines_annual_df.to_csv('chunks_by_lines_annual_df.csv', index=False)

print("chunks_by_lines_annual_df")
chunks_by_lines_annual_df

chunks_by_lines_annual_df


,groupe,code_h,ligne,valeurs
0,Chiffre d'affaire,1.,RECETTES,"{'Réel Annuelle 2022': '1856015 €', 'Réel Annu..."
1,Chiffre d'affaire,1.1.,Loyers logements et parkings HT,"{'Réel Annuelle 2022': '1715744 €', 'Réel Annu..."
2,Chiffre d'affaire,1.1.1.,CA Locatif Estudines,"{'Réel Annuelle 2022': '1709962 €', 'Réel Annu..."
3,Chiffre d'affaire,1.1.4.,CA Locatif Parkings,"{'Réel Annuelle 2022': '5782 €', 'Réel Annuell..."
4,Chiffre d'affaire,1.2.,RECETTES ANNEXES,"{'Réel Annuelle 2022': '129946 €', 'Réel Annue..."
...,...,...,...,...
146,Chiffre d'affaire,10.,% DES RECETTES TOTALES,"{'Réel Annuelle 2022': '2.11 %', 'Réel Annuell..."
147,Chiffre d'affaire,11.,Dotations aux amortissements (CAPEX),"{'Réel Annuelle 2022': '14788 €', 'Réel Annuel..."
148,Marge,12.,EBITDA,"{'Réel Annuelle 2022': '54004 €', 'Réel Annuel..."
149,Charge,13.,CAPEX,"{'Réel Annuelle 2022': '95452 €', 'Réel Annuel..."


In [13]:
# ------------------------
# CHUNKS PAR LIGNE MENSUELLE
# ------------------------
chunks_by_lines_mensual = []
cols = sorted(set([col for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple) and col[-1] == "Mensuelle"]))
for _, row in df_pivot_sorted.iterrows():
    doc = {
        "groupe": row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"],
        "code_h": row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"],
        "ligne": row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"],
        "valeurs": (
            {
                (
                    f"{c[1]} {'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()[c[2]-1] if isinstance(c[2], int) and 1 <= c[2] <= 12 else c[2]} {c[0]}"
                ):
                (
                    "{:.0f} €".format(v.iloc[0] if hasattr(v, "iloc") else v)
                    if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                    else "N/A"
                )
                for c in cols
                for v in [row.get(c)]
            }
            if not str(row["Ligne_Analytique"].iloc[-1]).lstrip().startswith('%')
            else
            {
                (
                    f"{c[1]} {'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()[c[2]-1] if isinstance(c[2], int) and 1 <= c[2] <= 12 else c[2]} {c[0]}"
                ):
                (
                    "{:.2f} %".format(v.iloc[0] if hasattr(v, "iloc") else v)
                    if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                    else "N/A"
                )
                for c in cols
                for v in [row.get(c)]
            }
        )
    }
    chunks_by_lines_mensual.append(doc)

chunks_by_lines_mensual_df = pd.DataFrame(chunks_by_lines_mensual)
chunks_by_lines_mensual_df.to_csv('chunks_by_lines_mensual_df.csv', index=False)

print("chunks_by_lines_mensual_df")
chunks_by_lines_mensual_df


chunks_by_lines_mensual_df


,groupe,code_h,ligne,valeurs
0,Chiffre d'affaire,1.,RECETTES,"{'Réel Jan 2022': '166643 €', 'Réel Feb 2022':..."
1,Chiffre d'affaire,1.1.,Loyers logements et parkings HT,"{'Réel Jan 2022': '143177 €', 'Réel Feb 2022':..."
2,Chiffre d'affaire,1.1.1.,CA Locatif Estudines,"{'Réel Jan 2022': '142098 €', 'Réel Feb 2022':..."
3,Chiffre d'affaire,1.1.4.,CA Locatif Parkings,"{'Réel Jan 2022': '1078 €', 'Réel Feb 2022': '..."
4,Chiffre d'affaire,1.2.,RECETTES ANNEXES,"{'Réel Jan 2022': '23467 €', 'Réel Feb 2022': ..."
...,...,...,...,...
146,Chiffre d'affaire,10.,% DES RECETTES TOTALES,"{'Réel Jan 2022': '23.06 %', 'Réel Feb 2022': ..."
147,Chiffre d'affaire,11.,Dotations aux amortissements (CAPEX),"{'Réel Jan 2022': '849 €', 'Réel Feb 2022': '7..."
148,Marge,12.,EBITDA,"{'Réel Jan 2022': '39274 €', 'Réel Feb 2022': ..."
149,Charge,13.,CAPEX,"{'Réel Jan 2022': '0 €', 'Réel Feb 2022': '0 €..."


In [14]:
# ------------------------
# 3. CHUNKS PAR ANNÉE - RESULTAT: annee code ligne valeur_reel valeur_previsionnel valeur_budget
# ------------------------

results = []
annees = sorted(set([col[0] for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple)]))

for an in annees:
    # On ne parcourt qu'une fois par ligne (pas pour chaque colonne/clé contexte)
    for _, row in df_pivot_sorted.iterrows():
        code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
        ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]

        # Extraire les colonnes pour chaque contexte à cette année
        contexte_map = {"Réel": "valeur_reel", "Prévision": "valeur_previsionnel", "Budget": "valeur_budget"}
        valeurs = {"valeur_reel": None, "valeur_previsionnel": None, "valeur_budget": None}

        for contexte_key, val_col in contexte_map.items():
            # Prendre toutes les colonnes correspondant à an, ce contexte, mois=12, Annuelle
            matching_cols = [col for col in df_pivot_sorted.columns[3:]
                                if isinstance(col, tuple)
                                and col[0] == an
                                and str(col[1]).lower().startswith(contexte_key.lower()[0])  # robust: R/B/P
                                and (len(col) > 2 and col[2] == 12)
                                and (len(col) > 3 and str(col[3]).lower() == "annuelle")]
            value_found = "N/A"
            for c in matching_cols:
                v = row.get(c)
                if not str(ligne).lstrip().startswith('%'):
                    value_found = (
                        "{:.0f} €".format(v.iloc[0] if hasattr(v, "iloc") else v)
                        if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                        else "N/A"
                    )
                else:
                    value_found = (
                        "{:.2f} %".format(v.iloc[0] if hasattr(v, "iloc") else v)
                        if pd.notnull(v.iloc[0] if hasattr(v, "iloc") else v)
                        else "N/A"
                    )
                # break on first
                break
            valeurs[val_col] = value_found

        results.append({
            "annee": an,
            "code": code_h,
            "ligne": ligne,
            "valeur_reel": valeurs["valeur_reel"],
            "valeur_previsionnel": valeurs["valeur_previsionnel"],
            "valeur_budget": valeurs["valeur_budget"]
        })

results_df = pd.DataFrame(results)
results_df.to_csv('results_by_year_with_contexts.csv', index=False)
print("results_df")
results_df

results_df


,annee,code,ligne,valeur_reel,valeur_previsionnel,valeur_budget
0,2022,1.,RECETTES,1856015 €,N/A,N/A
1,2022,1.1.,Loyers logements et parkings HT,1715744 €,N/A,N/A
2,2022,1.1.1.,CA Locatif Estudines,1709962 €,N/A,N/A
3,2022,1.1.4.,CA Locatif Parkings,5782 €,N/A,N/A
4,2022,1.2.,RECETTES ANNEXES,129946 €,N/A,N/A
...,...,...,...,...,...,...
750,2026,10.,% DES RECETTES TOTALES,N/A,N/A,13.40 %
751,2026,11.,Dotations aux amortissements (CAPEX),N/A,N/A,34447 €
752,2026,12.,EBITDA,N/A,N/A,332103 €
753,2026,13.,CAPEX,N/A,N/A,0 €


In [15]:
import ast

def parse_valeurs_field(val):
    """Try to parse the 'valeurs' field which looks like a stringified dict.
    Return a dict mapping keys to raw string values."""
    if pd.isna(val):
        return {}
    if isinstance(val, dict):
        return val
    try:
        # Some fields may already be like "{'k': 'v', ...}"
        parsed = ast.literal_eval(val)
        if isinstance(parsed, dict):
            return parsed
    except Exception:
        pass
    # fallback: try to extract "key': 'value" pairs with regex
    pattern = r"'([^']+)'\s*:\s*'([^']*)'"
    matches = re.findall(pattern, str(val))
    return {k:v for k,v in matches}

def numeric_from_string(s):
    """Extract a float from a string like '1 709 962 €' or '1709962' or '1,709,962'."""
    if s is None:
        return None
    if not isinstance(s, str):
        s = str(s)
    # remove euro sign and spaces/non-digit except comma and dot and minus
    cleaned = re.sub(r"[^\d\-,\.]", "", s)
    # replace comma used as thousand separator with nothing if dot exists or vice versa
    # handle common cases: '1 709 962' -> '1709962'
    cleaned = cleaned.replace(" ", "")
    # If there's both comma and dot, assume comma thousands -> remove commas
    if cleaned.count(",") > 0 and cleaned.count(".") > 0:
        cleaned = cleaned.replace(",", "")
    # If comma present and no dot, treat comma as decimal separator -> replace with dot
    if cleaned.count(",") > 0 and cleaned.count(".") == 0:
        cleaned = cleaned.replace(",", ".")
    try:
        return float(cleaned)
    except Exception:
        return None

def build_chunk(row, mensal_dict):
    groupe = str(row.get("groupe", ""))
    code_h = str(row.get("code_h", ""))
    ligne = str(row.get("ligne", ""))
    vals_ann = parse_valeurs_field(row.get("valeurs", {}))
    vals_mens = parse_valeurs_field(mensal_dict.get("valeurs", {})) if mensal_dict is not None else {}
    
    # Format annual and monthly sections in readable lines
    def format_dict(d):
        lines = []
        # sort keys naturally (year order) if possible
        for k in sorted(d.keys()):
            lines.append(f"- {k} : {d[k]}")
        return "\n".join(lines) if lines else "- Aucune donnée disponible"
    
    ann_text = format_dict(vals_ann)
    mens_text = format_dict(vals_mens)
    
    # Summary computations: try to detect last two annual numeric values for variation
    # extract keys that look like 'Réel Annuelle YYYY' or contain a year
    year_key_vals = []
    for k,v in vals_ann.items():
        m = re.search(r"(\d{4})", k)
        if m:
            year = int(m.group(1))
            num = numeric_from_string(v)
            if num is not None:
                year_key_vals.append((year, num, k))
    year_key_vals.sort()
    variation = "Aucune variation calculable"
    resume_principal = "Donnée principale non déterminée"
    points_attention = []
    if year_key_vals:
        # principal: most recent year value
        latest = year_key_vals[-1]
        resume_principal = f"{latest[2]} = {latest[1]:,.0f} €" if latest[1] is not None else f"{latest[2]} = {vals_ann.get(latest[2])}"
        if len(year_key_vals) >= 2:
            prev = year_key_vals[-2]
            if prev[1] is not None and latest[1] is not None and prev[1] != 0:
                pct = (latest[1]-prev[1]) / prev[1] * 100
                variation = f"{pct:+.1f}% entre {prev[0]} et {latest[0]}"
            else:
                variation = "Variation non calculable (valeurs manquantes ou nulles)"
    else:
        points_attention.append("Pas de données annuelles numériques détectées")
    
    # Points d'attention: missing months detection (expecting months keys like 'Réel Jan 2022')
    month_keys = [k for k in vals_mens.keys() if re.search(r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|Janv|Fev|Mar|Avr|Mai|Juin|Juil|Aout|Août|Sep|Oct|Nov|Dec)", k, re.IGNORECASE)]
    if month_keys:
        # crude check: count months for the most present year
        years = {}
        for k in month_keys:
            m = re.search(r"(\d{4})", k)
            if m:
                years.setdefault(m.group(1), 0)
                years[m.group(1)] += 1
        if years:
            # find year with max months
            max_year, count_months = max(years.items(), key=lambda x: x[1])
            if int(count_months) < 12:
                points_attention.append(f"{count_months} mois trouvés pour l'année {max_year} (mois manquants)")
    else:
        points_attention.append("Pas de données mensuelles détectées")
    
    # Compose chunk text
    chunk = (
        f"Groupe : {groupe}\n"
        f"Code Hiérarchie : {code_h}\n"
        f"Libellé : {ligne}\n\n"
        f"Données Annuelles :\n{ann_text}\n\n"
        f"Données Mensuelles :\n{mens_text}\n\n"
        f"Résumé :\n"
        f"- Donnée principale : {resume_principal}\n"
        f"- Variation notable : {variation}\n"
        f"- Points d'attention : {('; '.join(points_attention)) if points_attention else 'Aucun point d\'attention'}"
    )
    return chunk

# Load provided files
df_annual = chunks_by_lines_annual_df
df_mensual = chunks_by_lines_mensual_df

# Build an index for mensual by (groupe, code_h, ligne) for quick lookup
mens_index = {}
for _, r in df_mensual.iterrows():
    key = (str(r.get("groupe","")), str(r.get("code_h","")), str(r.get("ligne","")))
    mens_index[key] = r.to_dict()

chunks1 = []
metas1 = []
for _, r in df_annual.iterrows():
    key = (str(r.get("groupe","")), str(r.get("code_h","")), str(r.get("ligne","")))
    mens_row = mens_index.get(key)
    chunk_text = build_chunk(r.to_dict(), mens_row)
    chunks1.append(chunk_text)
    metas1.append({
        "groupe": r.get("groupe",""),
        "code_h": r.get("code_h",""),
        "ligne": r.get("ligne","")
    })

len(chunks1), len(metas1)

(151, 151)

In [16]:
chunks2 = []
metas2 = []

for _, row in results_df[["code", "ligne"]].iterrows():
    chunks2.append(f"Le code hierarchique de la ligne {row["ligne"]} est {row["code"]}")

for _, row in results_df[["code", "ligne"]].iterrows():
    metas2.append({"code_h": row["code"], "ligne": row["ligne"]})

len(chunks2), len(metas2)

(755, 755)

In [17]:
chunks3 = []
metas3 = []

chunks3.extend(chunks_df["text"].to_list())

def to_python_type(val):
    if isinstance(val, (np.integer,)):
        return int(val)
    elif isinstance(val, (np.floating,)):
        return float(val)
    elif isinstance(val, (np.bool_,)):
        return bool(val)
    elif isinstance(val, (np.ndarray, list, tuple)):
        return [to_python_type(x) for x in val]
    else:
        return val

for _, row in chunks_df.drop(columns=["text"]).iterrows():
    meta = {}
    for k, v in row.items():
        meta[str(k)] = to_python_type(v)
    metas3.append(meta)

len(chunks3), len(metas3)

(1812, 1812)

---

In [18]:
CHUNKS = []
METAS = []

CHUNKS.extend(chunks1)
CHUNKS.extend(chunks2)
CHUNKS.extend(chunks3)

METAS.extend(metas1)
METAS.extend(metas2)
METAS.extend(metas3)

len(CHUNKS), len(METAS)

(2718, 2718)

# Vectorisation et enregistrement dans chromadb

In [17]:
import chromadb
from sentence_transformers import SentenceTransformer

In [18]:
EMBEDDING_MODEL_NAME = 'distiluse-base-multilingual-cased-v2'
# Propositions de modèles adaptés pour de longues séquences (~1000 tokens) :
# - 'intfloat/e5-large-v2' (max_seq_length par défaut : 512, mais peut être poussé à 1024, supporte les longues séquences)
# - 'thenlper/gte-large' (rapide, supporte 1024 tokens)
# - 'BAAI/bge-large-en-v1.5' (peut être tuné pour 1024 tokens mais principale pour l'anglais)
# - 'flax-sentence-embeddings/all_datasets_v4_mpnet-base' (supporte 1024 tokens)
# - 'sentence-transformers/all-MiniLM-L12-v2' (moins puissant mais rapide; pour le multilingue: distiluse ou paraphrase-multilingual-mpnet-base-v2)

# Pour ~1000 tokens, la gamme e5/gte/BGE en "large" ou "xl" est recommandée si VRAM >= 8Go. 
# Exemple:
EMBEDDING_MODEL_NAME = "thenlper/gte-large"  # supporte 1024 tokens et rapide

embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)


In [19]:
from sentence_transformers.models import Transformer

# Pour certains modèles, il faut régler max_seq_length AVANT tout appel à encode()
# Si une erreur "The size of tensor a (927) must match the size of tensor b (512)" survient,
# assurez-vous que TOUS les sous-modules Transformer ont bien max_seq_length >= à la taille maximale de vos chunks (en tokens).
for mod in embed_model.modules():
    if isinstance(mod, Transformer):
        mod.auto_model.config.max_position_embeddings = 1024  # force le modèle sous-jacent à accepter 1024 tokens
        mod.max_seq_length = 1024

embed_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 1024, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [27]:
ids = [f"doc_{i}" for i in range(len(CHUNKS))]
len(ids)

2718

In [21]:
def clean_metadata_types(metadata_list):
    """
    Convertit récursivement les types NumPy (int64, np.float64, etc.) 
    en types Python natifs (int, float) dans une liste de dictionnaires.
    Ceci est essentiel pour la compatibilité avec ChromaDB.
    """
    cleaned_metadata = []
    for entry in metadata_list:
        cleaned_entry = {}
        for key, value in entry.items():
            # Conversion des types numériques NumPy
            if isinstance(value, (np.int64, np.int32, np.int8)):
                cleaned_entry[key] = int(value)
            elif isinstance(value, (np.float64, np.float32)):
                # Utiliser round() ou s'assurer que float(value) n'est pas NaN
                cleaned_entry[key] = float(value) if not np.isnan(value) else None
            elif isinstance(value, (int, float, str, bool, type(None))):
                cleaned_entry[key] = value
            else:
                # Gérer d'autres types non supportés si nécessaire (ex: convertir en str)
                cleaned_entry[key] = str(value)
        cleaned_metadata.append(cleaned_entry)
    return cleaned_metadata

METAS = clean_metadata_types(METAS)

In [22]:
def generate_embeddings(texts):
    """
    Génère les vecteurs à partir du texte, 
    en s'assurant que la longueur de chaque chunk en tokens ne dépasse pas le max_seq_length du modèle.
    """
    try:
        # Pour corriger l'erreur de dimension, il faut s'assurer que chaque chunk ne dépasse pas le max_seq_length et surtout que le modèle encode bien pour la taille attendue (ex: 512 tokens seulement pour certains modèles).
        from transformers import AutoTokenizer

        # Trouver le tokenizer
        try:
            tokenizer = embed_model.tokenizer
        except AttributeError:
            model_name = getattr(embed_model, 'name', None) or getattr(embed_model, 'model_card', None)
            if model_name is None:
                model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
            tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Il arrive que le modèle encode automatiquement à un max_seq_length plus petit (souvent 512 tokens pour Bert/MPNet, même si on met plus grand !)
        # On essaie de détecter la vraie taille max du modèle
        if hasattr(embed_model, "max_seq_length"):
            max_length = embed_model.max_seq_length
        elif hasattr(embed_model, "auto_model") and hasattr(embed_model.auto_model.config, "max_position_embeddings"):
            max_length = embed_model.auto_model.config.max_position_embeddings
        elif hasattr(tokenizer, "model_max_length"):
            max_length = tokenizer.model_max_length
        else:
            max_length = 512  # safe default

        # Certains modèles lèvent l'erreur si la taille du batch ou du chunk dépasse 512, même si max_seq_length=1024 ci-dessus
        # Donc tronquons systématiquement à la plus petite valeur pertinente
        max_model_len = min(512, max_length)

        truncated_texts = []
        for txt in texts:
            tokens = tokenizer.encode(txt, max_length=max_model_len, truncation=True, add_special_tokens=True)
            decoded_txt = tokenizer.decode(tokens, skip_special_tokens=True)
            truncated_texts.append(decoded_txt)

        return embed_model.encode(
            truncated_texts,
            batch_size=64,
            convert_to_numpy=True,
            show_progress_bar=True,
        ).tolist()
    except Exception as e:
        print(e)
    
embeddings = []
embeddings = generate_embeddings(CHUNKS)

Batches: 100%|██████████| 43/43 [09:22<00:00, 13.07s/it] 


In [23]:
len(embeddings)

2718

In [28]:
def indexer_chromadb(embeddings, chunks, metadata, ids):
    """Indexe les chunks dans ChromaDB."""
    try:
        # --- Configuration ChromaDB ---
        client = chromadb.Client()
        collection_name = 'residence_finance_analysis_complete'

        # Suppression pour un ré-indexage propre
        try:
            client.delete_collection(name=collection_name)
        except Exception:
            pass # Ignore si la collection n'existe pas

        collection = client.get_or_create_collection(
            name=collection_name, 
            metadata={"hnsw:space": "cosine"}
        )

        # Ajout des documents à la collection
        collection.add(
            embeddings=embeddings,
            documents=chunks,
            metadatas=metadata,
            ids=ids
        )

        print(f"\n✅ Indexation complète. {collection.count()} documents analytiques sont maintenant disponibles dans ChromaDB.")
        print("La base est prête pour l'interrogation par Ollama via RAG.")
    except Exception as e:
        print(f"Erreur lors de l'indexation: {e}")

# Lancement de l'indexation
indexer_chromadb(embeddings, CHUNKS, METAS, ids)


✅ Indexation complète. 2718 documents analytiques sont maintenant disponibles dans ChromaDB.
La base est prête pour l'interrogation par Ollama via RAG.


---

In [29]:
from langchain_ollama  import OllamaLLM
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [64]:
# --- Configuration Globale ---
# Nom de la collection ChromaDB précédemment créée
COLLECTION_NAME = 'residence_finance_analysis_complete'
# Modèle Ollama à utiliser (doit être téléchargé et lancé via 'ollama run <modele>')
OLLAMA_MODEL = config.GPT #  Choisissez votre modèle Ollama (ex: llama3:8b, gpt-oss)
OLLAMA_BASE_URL = "http://si-5/" # URL par défaut d'Ollama
# Seuil de récupération (Top K)
K_RETRIEVAL = 10

In [ ]:
from sentence_transformers import util

class SentenceTransformerEmbeddings:
    """Wrapper simple pour LangChain sur SentenceTransformer."""
    def embed_documents(self, texts):
        return embed_model.encode(texts).tolist()
    def embed_query(self, text):
        return embed_model.encode(text).tolist()

# 2. Chargement de la Base Vectorielle (ChromaDB)
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=SentenceTransformerEmbeddings(),
)

# Reranker utilisant le modèle de similarité du modèle d'embedding
class SimpleReranker:
    """
    Rerankeur post-retrieval basé sur la similarité cosine via SentenceTransformer.
    """
    def __init__(self, embedder):
        self.embedder = embedder

    def rerank(self, query, docs, top_n=None):
        # docs = liste de chaînes (contenu chunk)
        query_emb = self.embedder.encode([query])[0]
        doc_embs = self.embedder.encode(docs)
        sims = util.cos_sim(query_emb, doc_embs)[0].cpu().numpy()
        ranked = sorted(zip(docs, sims), key=lambda x: x[1], reverse=True)
        if top_n:
            ranked = ranked[:top_n]
        return [doc for doc, sim in ranked]

# 3. Création du Retriever
# Le retriever est l'outil qui recherche les chunks les plus pertinents (Top K)
retriever = vectorstore.as_retriever(search_kwargs={"k": K_RETRIEVAL})

# Ajout du reranker sur la sortie du retriever
reranker = SimpleReranker(embed_model)

def retrieve_and_rerank(query, final_top_k=5):
    # Récupère les chunks initiaux via le retriever
    doc_objs = retriever.invoke(query)
    docs = [d.page_content if hasattr(d, "page_content") else d for d in doc_objs]
    reranked_docs = reranker.rerank(query, docs, top_n=final_top_k)
    return reranked_docs

print(f"✅ Retriever ChromaDB initialisé pour récupérer les Top {K_RETRIEVAL} chunks + rerank avec reranker basé sur la similarité.")

✅ Retriever ChromaDB initialisé pour récupérer les Top 10 chunks + rerank avec reranker basé sur la similarité.


In [32]:
# 1. Connexion au modèle Ollama
llm = OllamaLLM(model=OLLAMA_MODEL, base_url=OLLAMA_BASE_URL)
print(f"✅ Connexion à Ollama ({OLLAMA_MODEL}) établie.")

✅ Connexion à Ollama (gpt-oss:20b) établie.


In [75]:
# 1. Template de Prompt (Instructions pour Ollama)
template = """
Tu es un Contrôleur de Gestion français expert et rigoureux pour des résidences.
Ta mission est d'analyser les données budgétaires et les écarts fournis ci-dessous dans le 'CONTEXTE FACTUEL'.
Réponds à la question de manière factuelle, précise et professionnelle.
Si les informations demandées ne sont pas dans le contexte, indique poliment que tu ne peux pas répondre.

CONTEXTE FACTUEL:
{context}

QUESTION:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Construction de la Chaîne RAG (LangChain Expression Language - LCEL)
# La chaîne suit le flux : Question -> Récupération des chunks -> Formatage du Prompt -> Appel à Ollama

rag_chain = (
    {"context": retrieve_and_rerank, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() # Convertit la sortie du LLM en simple chaîne de caractères
)

In [76]:
# --- Exemple de Requête ---
query_1 = "Quelles sont les recettes réelles de la résidence en 2024"

# Exécution de la chaîne
try:
    response = rag_chain.invoke(query_1)
    
    print("="*70)
    print(f"QUESTION: {query_1}")
    print("="*70)
    html_text = format_response(response)
    display(HTML(html_text))

except Exception as e:
    print(f"\nUne erreur est survenue lors de l'appel RAG : {e}")
    print("Vérifiez que le serveur Ollama est démarré et que le modèle spécifié est chargé.")

QUESTION: Quelles sont les recettes réelles de la résidence en 2024


---

![datacamp example](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/jumpstart/jumpstart-fm-rag.jpg)

![datacamp example](https://media.datacamp.com/legacy/v1704459771/image_552d84ab56.png)

!["GitHUB image"](https://private-user-images.githubusercontent.com/93597510/391476171-508b3a87-ac46-4bf7-b849-145c5465a6c0.png?jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NjUyODA2NTYsIm5iZiI6MTc2NTI4MDM1NiwicGF0aCI6Ii85MzU5NzUxMC8zOTE0NzYxNzEtNTA4YjNhODctYWM0Ni00YmY3LWI4NDktMTQ1YzU0NjVhNmMwLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNTEyMDklMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUxMjA5VDExMzkxNlomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTFlMzE3ODdjZTI4ZThlNzhjNzNhNDgxYTBjODlhZjExMTgxZDg1MDEwYjM0YWIzNWVjMWFhMzI2NGI1YWIyZjMmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0In0.9dXUUdR5kaivx6XICFKX5-nsi_CmwlBPWzir_EN12Hg)

---

In [ ]:
class OllamaClient:
    def __init__(self, ollama_url: str):
        self.client = httpx.AsyncClient(timeout=httpx.Timeout(595))
        self.url = ollama_url

    async def ask_ollama(self, prompt: str):
        messages = [{"role": "user", "content": prompt}]
        payload = {
            "model": config.GPT, # gpt-oss:20b
            "messages": messages,
            "stream": False,
            "keep_alive": -1,
            "options": {
                "temperature": 0.1,   # réponse très déterministe
            }
        }

        response = await self.client.post(
            url=self.url,
            json=payload
        )
        response.raise_for_status()
        json_data = response.json()
        if "message" in json_data and "content" in json_data["message"]:
            content: str = json_data["message"]["content"]

            parts = content.split("</think>", 1)
            if len(parts) > 1:
                content = parts[1]
            else:
                content = content
            html_text = format_response(content)
        return html_text, content

ollamaClient = OllamaClient(config.OLLAMA_URL)

In [30]:
PROMPT = f"""Tu es analyste financier senior spécialisé en exploitation immobilière (résidences étudiantes).

DONNÉES FOURNIES (Tableaux de bord opérationnels)
{df_pivot_sorted_annual_h1.to_markdown(index = False)}

OBJECTIF
Produis une analyse courte, actionnable et strictement basée sur les données ci-dessus, destinée à un directeur d'exploitation ou financier non spécialiste technique.

RAPPELS IMPORTANTS
- Langue : français. Ton professionnel, clair, phrases courtes.
- Aucune invention : si une information manque, écris exactement "Non disponible".
- Toujours citer les montants exacts avec unité (€, %, année).
- Ne produis PAS de JSON, PAS de code, PAS de balises. Sortie en TEXTE simple.
- Pas de retour à la ligne inutile.
- Sections limitées : 4-6 puces ou 5 lignes max.

FORMAT DE SORTIE EXACT (RESPECTE STRICTEMENT LES TITRES SUIVANTS)

Résumé (5 lignes max)
- [Phrase d'ouverture décrivant la tendance générale de l'exploitation]
- [1-2 faits clés chiffrés : ex. évolution du CA, écart Budget/Réel, taux d'occupation]
- [1 phrase sur les charges ou marges, appuyée d'un chiffre]
- [1 phrase sur la stabilité ou dispersion des performances]
- [Conclusion courte sur la situation opérationnelle]

KPIs clés (3-5 puces, chiffres exacts)
- Chiffre d'affaires : [valeur € + période]
- Charges d'exploitation : [valeur € + période]
- Résultat net ou marge opérationnelle : [valeur € ou %]
- Écarts Réel vs Budget : [valeur € et %]
- Mets "Non disponible" si un KPI manque.

Risques (2-3 puces, 1 ligne chacun, appuyés par un chiffre)
- [Risque synthétique] — évidence : [indicateur = valeur + période]

Opportunités (2-3 puces, 1 ligne chacun, appuyées par un chiffre)
- [Opportunité synthétique] — évidence : [indicateur = valeur + période]

Actions proposées (3 puces, concrètes et exécutables)
- [Action #1 courte] — fondée sur [indicateur + chiffre]
- [Action #2 courte] — fondée sur [indicateur + chiffre]
- [Action #3 courte] — fondée sur [indicateur + chiffre]

Notes sur les données (optionnel, 2 puces max)
- [Ex. valeurs manquantes / anomalies : "Non disponible" pour …]
- [Ex. limites d'interprétation : ex. certaines années incomplètes]

CONTRAINTE FINALE
- Réponds UNIQUEMENT en TEXTE selon la structure ci-dessus. AUCUN JSON, AUCUN CODE, AUCUNE BALISE.
"""

In [31]:
def format_value(val):
    try:
        if isinstance(val, str) and val.strip().startswith("%"):
            num = float(val.strip().replace("%", "").replace(",", "."))
            return "{:.2f} %".format(num)

        if isinstance(val, str) and "%" in val:
            num = float(val.replace("%", "").replace(",", ".").strip())
            return "{:.2f} %".format(num)

        if isinstance(val, (float, np.floating, int, np.integer)):
            if float(val) == int(val):
                return int(val)
            else:
                return "{:.2f}".format(float(val))

        if isinstance(val, str):
            num = float(val.replace(",", ".").strip())
            if num == int(num):
                return int(num)
            else:
                return "{:.2f}".format(num)
        return val
    except:
        return val

for col in df_pivot_sorted_annual_h1.columns:
    df_pivot_sorted_annual_h1[col] = df_pivot_sorted_annual_h1[col].apply(format_value)

df_pivot_sorted_annual_h1

Annee                            Groupe                      Ligne_Analytique  \
Contexte                                                                        
Mois                                                                            
Nature de l'écriture                                                            
0                     Chiffre d'affaire                              RECETTES   
1                                Charge           CHARGES D'IMMEUBLE DIRECTES   
2                                 Marge                               MARGE 1   
3                     Chiffre d'affaire                % DES RECETTES TOTALES   
4                                Charge                               Total 1   
5                                 Marge                               MARGE 2   
6                     Chiffre d'affaire                % DES RECETTES TOTALES   
7                                Charge                               Total 2   
8                                 Marge                               MARGE 3   
9                     Chiffre d'affaire                % DES RECETTES TOTALES   
10                    Chiffre d'affaire  Dotations aux amortissements (CAPEX)   
11                                Marge                                EBITDA   
12                               Charge                                 CAPEX   
13                                Marge                        FREE CASH FLOW   

Annee                    2024      2025              2026  
Contexte                 Réel Prévision   Budget   Budget  
Mois                       12        12       12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle Annuelle  
0                     2193409   2182306  2205868  2221683  
1                      663916    666871   650619   602278  
2                     1529494   1515435  1555249  1619405  
3                       69.73     69.44    70.51    72.89  
4                     1088310   1115597  1128644  1125933  
5                      441184    399838   426605   493472  
6                       20.11     18.32    19.34    22.21  
7                      216976    218108   204336   195815  
8                      224208    181730   222269   297657  
9                       10.22      8.33    10.08    13.40  
10                      28095     23186    27076    34447  
11                     252303    204917   249344   332103  
12                      15964     19270        0        0  
13                     236339    185646   249344   332103

In [33]:
html_text = await ollamaClient.ask_ollama(PROMPT)
display(HTML(html_text))

---

In [23]:
df_pivot_sorted_annual_h2_charge = df_pivot_sorted_annual_h2[df_pivot_sorted_annual_h2["Groupe"] == "Charge"].drop(columns=["Groupe"]).reset_index(drop=True)

for col in df_pivot_sorted_annual_h2_charge.columns:
    df_pivot_sorted_annual_h2_charge[col] = df_pivot_sorted_annual_h2_charge[col].apply(format_value)

df_pivot_sorted_annual_h2_charge

C:\Users\Amsata\AppData\Local\Temp\ipykernel_20048\2769236681.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_pivot_sorted_annual_h2_charge = df_pivot_sorted_annual_h2[df_pivot_sorted_annual_h2["Groupe"] == "Charge"].drop(columns=["Groupe"]).reset_index(drop=True)


Annee                Code_H                              Ligne_Analytique  \
Contexte                                                                    
Mois                                                                        
Nature de l'écriture                                                        
0                         2                   CHARGES D'IMMEUBLE DIRECTES   
1                      2.1.                            FRAIS DE PERSONNEL   
2                      2.2.                      IMPOTS et TAXES DIVERSES   
3                      2.3.                                       ENERGIE   
4                      2.4.                       CONTRATS DE MAINTENANCE   
5                      2.6.             REMISE EN ETAT PARTIES PRIVATIVES   
6                      2.7.               REMISE EN ETAT PARTIES COMMUNES   
7                      2.8.                         LINGE ET BLANCHISSAGE   
8                      2.9.                    FOURNITURES D'EXPLOITATION   
9                     2.10.                         ACHAT ALIMENTAIRE PDJ   
10                    2.12.            LOCATIONS, HONORAIRES, SACEM ET TV   
11                    2.13.                        DEPLACEMENT ET MISSION   
12                    2.14.                                     PUBLICITE   
13                    2.15.                            Publicité Internet   
14                    2.16.                   FOURNITURES ADMINISTRATIVES   
15                    2.17.          TELEPHONIE - INTERNET - INFORMATIQUE   
16                        5                                       Total 1   
17                     5.1.                  AMORTISSEMENT et CREDIT BAIL   
18                     5.2.                 LOYERS REVERSES PROPRIETAIRES   
19                     5.3.                                  GROS TRAVAUX   
20                     5.4.                               INCIDENCE T.V.A   
21                     5.5.                             CET (ex Taxe Pro)   
22                        8                                       Total 2   
23                     8.1.  PRESTATIONS INTERNES (Y/C Honoraires syndic)   
24                     8.2.                            FRAIS DES SOCIETES   
25                     8.3.                             HONORAIRES SYNDIC   
26                     8.4.                       CHARGES EXCEPTIONNELLES   
27                       13                                         CAPEX   

Annee                    2022     2023     2024      2025              2026  
Contexte                 Réel     Réel     Réel Prévision   Budget   Budget  
Mois                       12       12       12        12       12       12  
Nature de l'écriture Annuelle Annuelle Annuelle  Annuelle Annuelle Annuelle  
0                      674637   613230   663916    666871   650619   602278  
1                      161106   163977   168150    196281   202380   145653  
2                       27062     3995     5502      5653     6452     5766  
3                      223806   236163   339699    269587   273248   274429  
4                       42782    53107    48597     67030    93021    69291  
5                      148835    74058    32530     21994    14480    35000  
6                        5952    17505    24998     45476        0     5000  
7                        3734     4359     3350      2718     3602     2435  
8                       19855    13309     3022      7052     2948     6291  
9                           0        0        0         0        0        0  
10                       6319     8613     4647     13217    11866    15113  
11                        905     2072     1120      1603     3124     1635  
12                       4762     2903     3011      2874     5327     8071  
13                       3122     2857     3063      4452     5781     4056  
14                        976     1881      826      1283      900     1272  
15                      25421    28433    25401     2

In [34]:
PROMPT_CHARGE = f""" Tu es analyste financier senior spécialisé en exploitation immobilière (résidences étudiantes).

DONNÉES
{df_pivot_sorted_annual_h2_charge.to_markdown(index = False)}

OBJECTIF
Produis une analyse ultra-claire, concise et strictement basée sur les données ci-dessus, destinée à un directeur d'exploitation.

Ton analyse doit identifier :
- les dynamiques des charges
- les postes qui augmentent/baisse significativement
- les risques opérationnels
- les leviers d'optimisation immédiats

RÈGLES
- Langue : français, ton professionnel simple.
- Aucune invention : si une information est absente, écrire « Non disponible ».
- Toujours citer les montants exacts avec unité (€, %, année).
- 5 lignes max par section.
- Pas de JSON, pas de code, pas de balises.
- Format texte uniquement.

FORMAT DE SORTIE

Résumé (5 lignes max)
- [Tendance générale des charges sur les années]
- [Variation significative : ex. +X % entre 2023 et 2024]
- [Stabilité ou volatilité des principaux blocs]
- [Conclusion courte]

KPIs clés (4-6 puces)
- Charges directes : [montant + période]
- Charges totales (Total 1 / Total 2) : [montant + période]
- CAPEX : [montant + période]
- Écart Prévision vs Budget : [valeur € et %]
- Autres KPIs pertinents : [montants + périodes]

Risques (2-3 puces)
- [Risque synthétique] — évidence : [indicateur = valeur + période]

Opportunités (2-3 puces)
- [Opportunité synthétique] — évidence : [indicateur = valeur + période]

Actions proposées (3 puces, concrètes et exécutables)
- [Action #1] — fondée sur [valeur + période]
- [Action #2] — fondée sur [valeur + période]
- [Action #3] — fondée sur [valeur + période]

Notes sur les données (optionnel)
- [Anomalies, valeurs manquantes, incohérences]
"""

In [ ]:
html_text = await ollamaClient.ask_ollama(PROMPT_CHARGE)
display(HTML(html_text))

---

In [6]:
df_pivot_sorted_annual_h1 = pd.read_pickle("df_pivot_sorted_annual_h1.pkl")

In [7]:
def format_value(val):
    try:
        if isinstance(val, str) and val.strip().startswith("%"):
            num = float(val.strip().replace("%", "").replace(",", "."))
            return "{:.2f} %".format(num)

        if isinstance(val, str) and "%" in val:
            num = float(val.replace("%", "").replace(",", ".").strip())
            return "{:.2f} %".format(num)

        if isinstance(val, (float, np.floating, int, np.integer)):
            if float(val) == int(val):
                return int(val)
            else:
                return "{:.2f}".format(float(val))

        if isinstance(val, str):
            num = float(val.replace(",", ".").strip())
            if num == int(num):
                return int(num)
            else:
                return "{:.2f}".format(num)
        return val
    except:
        return val

for col in df_pivot_sorted_annual_h1.columns:
    df_pivot_sorted_annual_h1[col] = df_pivot_sorted_annual_h1[col].apply(format_value)

df_pivot_sorted_annual_h1

Annee                            Groupe                      Ligne_Analytique  \
Contexte                                                                        
Mois                                                                            
Nature de l'écriture                                                            
0                     Chiffre d'affaire                              RECETTES   
1                                Charge           CHARGES D'IMMEUBLE DIRECTES   
2                                 Marge                               MARGE 1   
3                     Chiffre d'affaire                % DES RECETTES TOTALES   
4                                Charge                               Total 1   
5                                 Marge                               MARGE 2   
6                     Chiffre d'affaire                % DES RECETTES TOTALES   
7                                Charge                               Total 2   
8                                 Marge                               MARGE 3   
9                     Chiffre d'affaire                % DES RECETTES TOTALES   
10                    Chiffre d'affaire  Dotations aux amortissements (CAPEX)   
11                                Marge                                EBITDA   
12                               Charge                                 CAPEX   
13                                Marge                        FREE CASH FLOW   

Annee                    2024      2025              2026  
Contexte                 Réel Prévision   Budget   Budget  
Mois                       12        12       12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle Annuelle  
0                     2193409   2182306  2205868  2221683  
1                      663916    666871   650619   602278  
2                     1529494   1515435  1555249  1619405  
3                       69.73     69.44    70.51    72.89  
4                     1088310   1115597  1128644  1125933  
5                      441184    399838   426605   493472  
6                       20.11     18.32    19.34    22.21  
7                      216976    218108   204336   195815  
8                      224208    181730   222269   297657  
9                       10.22      8.33    10.08    13.40  
10                      28095     23186    27076    34447  
11                     252303    204917   249344   332103  
12                      15964     19270        0        0  
13                     236339    185646   249344   332103

In [10]:
df_pivot_sorted_annual_h1_the_year = df_pivot_sorted_annual_h1.drop(columns=[col for col in df_pivot_sorted_annual_h1.columns[2:] if (col[0] != datetime.now().year and col[0] != datetime.now().year-1)])
df_pivot_sorted_annual_h1_the_year

Annee                            Groupe                      Ligne_Analytique  \
Contexte                                                                        
Mois                                                                            
Nature de l'écriture                                                            
0                     Chiffre d'affaire                              RECETTES   
1                                Charge           CHARGES D'IMMEUBLE DIRECTES   
2                                 Marge                               MARGE 1   
3                     Chiffre d'affaire                % DES RECETTES TOTALES   
4                                Charge                               Total 1   
5                                 Marge                               MARGE 2   
6                     Chiffre d'affaire                % DES RECETTES TOTALES   
7                                Charge                               Total 2   
8                                 Marge                               MARGE 3   
9                     Chiffre d'affaire                % DES RECETTES TOTALES   
10                    Chiffre d'affaire  Dotations aux amortissements (CAPEX)   
11                                Marge                                EBITDA   
12                               Charge                                 CAPEX   
13                                Marge                        FREE CASH FLOW   

Annee                    2024      2025           
Contexte                 Réel Prévision   Budget  
Mois                       12        12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle  
0                     2193409   2182306  2205868  
1                      663916    666871   650619  
2                     1529494   1515435  1555249  
3                       69.73     69.44    70.51  
4                     1088310   1115597  1128644  
5                      441184    399838   426605  
6                       20.11     18.32    19.34  
7                      216976    218108   204336  
8                      224208    181730   222269  
9                       10.22      8.33    10.08  
10                      28095     23186    27076  
11                     252303    204917   249344  
12                      15964     19270        0  
13                     236339    185646   249344

In [11]:
def _format_euro_fr(x: float, line: str) -> str:
    """Format number in French style with 2 decimals and a non-breaking space thousands separator."""
    if pd.isna(x):
            return "N/A"
    elif str(line).startswith("%"):
        s = f"{x:,.2f}"
        s = s.replace(",", " ")
        return f"{s} %"
    else:
        s = f"{x:,.0f}"
        s = s.replace(",", " ")
        return f"{s} €"

def _normalize_col(col):
    if isinstance(col, tuple):
        if len(col) >= 1:
            year = str(col[0])
        else:
            year = "unknown"
        # find Réel/Prévision/Budget if present in tuple (also accent-insensitive, lowercase!)
        typ = next(
            (str(x) for x in col if isinstance(x, str) and str(x).lower() in ("réel", "budget", "prévision", "prevision")),
            None
        )
        if typ is None:
            # search for any string in tuple as fallback
            typ = next((str(x) for x in col if isinstance(x, str)), "Réel")
        return {"year": year, "type": typ}
    else:
        # If col is not a tuple, fallback as string
        year = str(col)
        return {"year": year, "type": "Réel"}

def transform_for_llm(df_pivot: pd.DataFrame):
    """
    Returns:
        - df_long: DataFrame with columns ['Indicateur','Année','Type','Valeur']
        - metric_text: str with metric blocks optimized for LLM input (with custom column order)
        - metric_json: dict structured as {INDICATOR: {period_key: numeric_value}}
    """
    df = df_pivot.copy()

    # Ensure index is a normal column named 'Indicateur' if index holds labels
    if df.index.name is None or df.index.name == "":
        df = df.reset_index()
    # Try to detect indicator column name
    indicator_col = None
    for possible in ['Ligne_Analytique', 'Indicateur', 'index', 0]:
        if possible in df.columns:
            indicator_col = possible
            break
    if indicator_col is None:
        indicator_col = df.columns[0]
    df = df.rename(columns={indicator_col: 'Indicateur'})

    value_cols = [c for c in df.columns if c != 'Indicateur']

    rows = []
    for _, row in df.iterrows():
        indicator = row['Indicateur']
        for col in value_cols:
            meta = _normalize_col(col)
            year = meta['year']
            typ = meta['type']
            try:
                val = row[col]
            except Exception:
                val = row.get(col, None)
            # Try to keep numeric
            numeric = None
            if pd.api.types.is_numeric_dtype(type(val)):
                try:
                    numeric = float(val) if not pd.isna(val) else None
                except Exception:
                    numeric = None
            else:
                try:
                    numeric = float(str(val).replace("€", "").replace("%", "").replace(" ", "").replace(",", "."))
                except Exception:
                    numeric = None
            label = "  ".join([str(x).strip() for x in indicator if x is not None])
            rows.append({
                'Indicateur': indicator,
                'Année': year,
                'Type': typ,
                'Valeur_num': numeric,
                'Valeur_txt': _format_euro_fr(numeric, label) if numeric is not None else "N/A"
            })

    df_long = pd.DataFrame(rows)
    #--- Custom LLM string format ---

    # Define context order mapping (for sorting)
    def context_rank(typ):
        t = str(typ).lower()
        if "réel" in t:
            return 0
        if "prevision" in t or "prévision" in t:
            return 1
        if "budget" in t:
            return 2
        return 99

    # Custom block for indicator
    def _block_for_indicator(ind):
        if isinstance(ind, (list, tuple, pd.Series, np.ndarray)):
            # Remove "Name" and dtype printing, get just the values
            label = "  ".join([str(x).strip() for x in ind if x is not None])
        else:
            label = str(ind).strip()
        lines = [f"[{label}]"]
        sub = df_long[df_long['Indicateur'].astype(str).values == str(ind)].copy()
        # Only keep valid numerical values; skip technical columns
        sub = sub[~((sub['Année'].str.lower() == 'groupe') | (sub['Année'].str.lower() == 'indicateur') | (sub['Année'].str.lower() == 'index'))]
        # Custom sort: context (réel - prévision - budget), then by year
        sub_sorted = sub.copy()
        try:
            sub_sorted["Année_num"] = pd.to_numeric(sub_sorted["Année"], errors='coerce')
        except Exception:
            sub_sorted["Année_num"] = sub_sorted["Année"]
        sub_sorted = sub_sorted.sort_values(by=["Année_num", "Type"], key=lambda col: col, ascending=[True, True])
        sub_sorted = sub_sorted.sort_values(
            by=["Type", "Année_num"],
            key=lambda col: col.map(context_rank) if col.name == "Type" else col,
            ascending=[True, True]
        )
        # Ensure correct order: Réel, Prévision, Budget
        # Gather existing contexts by year
        entries = []
        for ctx in ["Réel", "Prévision", "Budget"]:
            sub_ctx = sub_sorted[sub_sorted["Type"].str.lower().str.contains(ctx.lower())]
            entries.append(sub_ctx)
        merged = pd.concat(entries)
        # Remove duplicates in case some contexts are missing
        merged = merged.drop_duplicates(subset=["Année", "Type"])

        # Yield custom lines
        for _, rr in merged.iterrows():
            year = rr['Année']
            typ = rr['Type']
            txt = rr['Valeur_txt']
            # skip empty technicals, or keep only if year is really a year
            if (year is None or str(year).lower() in ['groupe', 'indicateur', 'index']) and (txt == "0" or txt == "0 €" or txt == "N/A"):
                continue
            # Always print 0, even for Groupe/Indicateur/Index, if present and value not 0?
            lines.append(f"- {year} {typ} : {txt}")
        return "\n".join(lines)

    # Only indicateur list (avoid 'Groupe Groupe', etc.)
    indicators = df_long['Indicateur'].drop_duplicates().tolist()
    blocks = [_block_for_indicator(ind) for ind in indicators]

    metric_text = "\n\n".join(blocks)

    return metric_text

metric_text = transform_for_llm(df_pivot_sorted_annual_h1_the_year)
print(count_tokens(metric_text))
print(metric_text)

378
[RECETTES]
- 2024 Réel : 2 193 409 €
- 2025 Prévision : 2 182 306 €
- 2025 Budget : 2 205 868 €

[CHARGES D'IMMEUBLE DIRECTES]
- 2024 Réel : 663 916 €
- 2025 Prévision : 666 871 €
- 2025 Budget : 650 619 €

[MARGE 1]
- 2024 Réel : 1 529 494 €
- 2025 Prévision : 1 515 435 €
- 2025 Budget : 1 555 249 €

[% DES RECETTES TOTALES]
- 2024 Réel : 69.73 %
- 2025 Prévision : 69.44 %
- 2025 Budget : 70.51 %

[Total 1]
- 2024 Réel : 1 088 310 €
- 2025 Prévision : 1 115 597 €
- 2025 Budget : 1 128 644 €

[MARGE 2]
- 2024 Réel : 441 184 €
- 2025 Prévision : 399 838 €
- 2025 Budget : 426 605 €

[% DES RECETTES TOTALES]
- 2024 Réel : 20.11 %
- 2025 Prévision : 18.32 %
- 2025 Budget : 19.34 %

[Total 2]
- 2024 Réel : 216 976 €
- 2025 Prévision : 218 108 €
- 2025 Budget : 204 336 €

[MARGE 3]
- 2024 Réel : 224 208 €
- 2025 Prévision : 181 730 €
- 2025 Budget : 222 269 €

[% DES RECETTES TOTALES]
- 2024 Réel : 10.22 %
- 2025 Prévision : 8.33 %
- 2025 Budget : 10.08 %

[Dotations aux amortissements (

In [84]:
PROMPT = f"""Tu es analyste financier senior spécialisé en exploitation immobilière (résidences étudiantes).

DONNÉES FOURNIES (Tableaux de bord opérationnels)
{metric_text}

RÈGLES D'ANALYSE
- Ne te base que sur les données présentes ci-dessus : aucune hypothèse, aucune estimation, aucune invention.
- Préfère toujours les dernières valeurs disponibles pour les comparaisons.
- Total 1 correspond Charges financières et immobilisations.
- Total 2 correspond Charges de gestion courante.
- Formules utilisées dans les données :
    • Marge 1 = Recettes - Charges d'immeuble directes.
    • Marge 2 = Marge 1 - Total 1.
    • Marge 3 = Marge 2 - Total 2.
    • 1e % des recettes totales = (Marge 1 / Recettes) * 100.
    • 2e % des recettes totales = (Marge 2 / Recettes) * 100.
    • 3e % des recettes totales = (Marge 3 / Recettes) * 100.
    • EBITDA = Marge 3 + Dotations aux amortissements.
    • FREE CASH FLOW = EBITDA - CAPEX.
- Si un indicateur manque ou est nul pour une année, indique « Non disponible ».

STYLE D'ÉCRITURE
- Langue : français professionnel.
- Phrases courtes, rigoureuses, orientées pilotage opérationnel.
- Pas d'introduction, pas de conclusion globale hors structure.
- Mentionne toujours années, valeurs et unités (€, %, écart).
- Sortie en TEXTE SIMPLE : aucun JSON, aucun code.

FORMAT DE SORTIE (À RESPECTER STRICTEMENT)

Résumé (5 lignes max)
- [Phrase d'ouverture indiquant la dynamique générale : évolution recettes/charges]
- [1 fait chiffré majeur sur recettes]
- [1 fait chiffré majeur sur les charges]
- [1 élément de stabilité ou tension sur les marges (%)]
- [Synthèse opérationnelle courte]

KPIs clés (3-5 puces)
- Chiffre d'affaires : [valeur + période]
- Charges d'exploitation : [valeur + période]
- EBITDA si disponible : sinon « Non disponible »
- Écarts Réel ou Prévision vs Budget : [écart en € et en %]

Risques (2-3 puces)
- [Risque identifié] — preuve : [indicateur = valeur + période]
- Formulation courte, orientée surveillance opérationnelle.

Opportunités (2-3 puces)
- [Opportunité identifiable] — preuve chiffrée : [indicateur = valeur + période]

Actions proposées (3 puces, exécutables)
- [Action #1] — appuyée par [chiffre clé]
- [Action #2] — appuyée par [chiffre clé]
- [Action #3] — appuyée par [chiffre clé]

Notes sur les données (optionnel, 2 puces)
- [Valeurs manquantes ou incohérences]
- [Rappel éventuel de limites d'interprétation]

CONTRAINTE FINALE
Réponds UNIQUEMENT selon la structure ci-dessus, en texte simple.
"""

In [85]:
count_tokens(PROMPT)

1011

In [92]:
html_text, brt = await ollamaClient.ask_ollama(PROMPT)
# display(HTML(html_text))
print(brt)

Résumé  
- Recettes stables, légère baisse 2025 puis hausse 2026 (+1,80 %).  
- Charges d’exploitation augmentent 2025 (+0,45 %) puis baissent 2026 (-9,68 %).  
- Marge 1 reste proche de 70 % mais Marge 2 chute 2025 (-9,37 %) avant de rebondir 2026 (+23,38 %).  
- Marge 3 et EBITDA connaissent une forte hausse 2026 (+63,86 % et +62,18 %).  
- Opérationnel : besoin d’optimiser les coûts et de soutenir la croissance de la marge.

KPIs clés  
- Chiffre d’affaires : 2024 : 2 193 409 € (Réel), 2025 : 2 182 306 € (Prévision), 2026 : 2 221 683 € (Budget).  
- Charges d’exploitation : 2024 : 663 916 € (Réel), 2025 : 666 871 € (Prévision), 2026 : 602 278 € (Budget).  
- EBITDA : 2024 : 252 303 € (Réel), 2025 : 204 917 € (Prévision), 2026 : 332 103 € (Budget).  
- Écarts Réel vs Budget : 2025 Prévision vs Budget – Recettes : -23 562 € (-1,07 %), Charges : +16 252 € (+2,50 %), EBITDA : -44 427 € (-17,82 %).

Risques  
- Marge 2 baisse 2025 – preuve : Marge 2 399 838 € vs 2024 441 184 € (diff -41 

In [24]:
PROMPT2 = f"""Tu es analyste financier senior spécialisé en exploitation immobilière (résidences étudiantes).

DONNÉES FOURNIES (Tableaux de bord opérationnels)
{metric_text}

RÈGLES D'ANALYSE
- Ne te base que sur les données présentes ci-dessus : aucune hypothèse, aucune estimation, aucune invention.
- Préfère toujours les dernières valeurs disponibles pour les comparaisons.
- Total 1 correspond Charges financières et immobilisations.
- Total 2 correspond Charges de gestion courante.
- Formules utilisées dans les données :
    • Marge 1 = Recettes - Charges d'immeuble directes.
    • Marge 2 = Marge 1 - Total 1.
    • Marge 3 = Marge 2 - Total 2.
    • 1e % des recettes totales = (Marge 1 / Recettes) * 100.
    • 2e % des recettes totales = (Marge 2 / Recettes) * 100.
    • 3e % des recettes totales = (Marge 3 / Recettes) * 100.
    • EBITDA = Marge 3 + Dotations aux amortissements.
    • FREE CASH FLOW = EBITDA - CAPEX.
- Analyse les écarts entre les valeurs Réel ou Prévision et Budget pour {datetime.now().year}.
- Fais également une analyse comparative entre Réel ou Prévision {datetime.now().year} et Réel {datetime.now().year-1}.
- Si un indicateur manque ou est nul pour une année, indique « Non disponible ».

STYLE D'ÉCRITURE
- Langue : français professionnel.
- Phrases courtes, rigoureuses, orientées pilotage opérationnel.
- Pas d'introduction, pas de conclusion globale hors structure.
- Mentionne toujours années, valeurs et unités (€, %, écart).
- Sortie en TEXTE SIMPLE : aucun JSON, aucun code.

FORMAT DE SORTIE

Résumé (5 lignes max)
[Phrase d'ouverture indiquant la dynamique générale : évolution recettes/charges]
[1 fait chiffré majeur sur recettes]
[1 fait chiffré majeur sur les charges]
[Synthèse opérationnelle courte]

KPIs clés (3 lignes)
- Écart Réel/Prévision vs Budget Chiffre d'affaires : [valeur précise, année, analyse d'écart synthétique]
- Écart Réel/Prévision vs Budget Charges d'exploitation : [valeur précise, année, analyse d'écart synthétique]
- Écart Réel/Prévision vs Budget EBITDA et FREE CASH FLOW : [valeur précise, année, analyse d'écart synthétique]

Risques (2-3 puces)
- [Risque identifié] — preuve : [indicateur = valeur + période]
- Formulation courte, orientée surveillance opérationnelle.

Opportunités (2-3 puces)
- [Opportunité identifiable] — preuve chiffrée : [indicateur = valeur + année]

Actions proposées (3 puces, concrètes, recommandées)
- [Action #1] — appuyée par [chiffre clé]
- [Action #2] — appuyée par [chiffre clé]
- [Action #3] — appuyée par [chiffre clé]

Notes sur le tableau de données (optionnel, 2 puces)
- [Valeurs manquantes ou incohérences]
- [Rappel éventuel de limites d'interprétation]

CONTRAINTE FINALE
Réponds selon la structure ci-dessus, en texte simple.
"""

count_tokens(PROMPT2)

940

In [26]:
display(HTML(html_text))

In [ ]:
display(HTML(html_text1))

In [27]:
html_text3, brt3 = await ollamaClient.ask_ollama(PROMPT2)
display(HTML(html_text3))